<a href="https://colab.research.google.com/github/hongan-z/_Mechine_Learning_ML/blob/main/cs179g_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 64.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=d8bafac1fa1dc6a15125881b3cc8e7b1388539d0bb9915a2d107800b494e686a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import pyspark
import pandas as pd
import json

In [3]:
with open("Clean_New_Data.json") as f:
    data = json.load(f)

In [4]:
df = pd.DataFrame(data)
df.head()

,title,genres,date,Metascore,Userscore,UserScorePositive,UserScoreMixed,UserScoreNegative
0,Three Colors: Red,"[Drama, Mystery, Romance]","November 23, 1994",100,8.6,152,4,9
1,Boyhood,[Drama],"July 11, 2014",100,7.5,"1,869",274,354
2,Casablanca,"[Drama, Romance, War]","January 23, 1943",100,8.9,360,11,10
3,Rear Window,"[Mystery, Thriller]","September 1, 1954",100,8.8,365,15,10
4,The Godfather,"[Drama, Thriller, Crime]","March 11, 1972",100,9.2,"3,293",93,115


In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Sc').getOrCreate()

In [6]:
spark


In [7]:
sparkdf =spark.createDataFrame(df) 
sparkdf.printSchema()
sparkdf.show()

root
 |-- title: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date: string (nullable = true)
 |-- Metascore: string (nullable = true)
 |-- Userscore: string (nullable = true)
 |-- UserScorePositive: string (nullable = true)
 |-- UserScoreMixed: string (nullable = true)
 |-- UserScoreNegative: string (nullable = true)

+--------------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+
|               title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|
+--------------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+
|   Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|      100|      8.6|              152|             4|                9|
|             Boyhood|             [Drama]|    July 11, 2014|      100|      7

In [8]:
type(df)

pandas.core.frame.DataFrame

In [9]:
type(sparkdf)

pyspark.sql.dataframe.DataFrame

In [10]:
type('UserScoreMixed') 

str

In [11]:
sparkdf.select('genres').show()

+--------------------+
|              genres|
+--------------------+
|[Drama, Mystery, ...|
|             [Drama]|
|[Drama, Romance, ...|
| [Mystery, Thriller]|
|[Drama, Thriller,...|
|    [Drama, History]|
|             [Drama]|
|[Drama, Comedy, R...|
|[Comedy, Romance,...|
|[Drama, Thriller,...|
|   [Comedy, Romance]|
|[Drama, Mystery, ...|
|[Adventure, Drama...|
|    [Drama, Mystery]|
|     [Drama, Comedy]|
|       [Documentary]|
|           [Western]|
|[Action, Drama, War]|
|[Drama, Comedy, R...|
|[Drama, Comedy, R...|
+--------------------+
only showing top 20 rows



In [12]:
sparkdf.show()

+--------------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+
|               title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|
+--------------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+
|   Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|      100|      8.6|              152|             4|                9|
|             Boyhood|             [Drama]|    July 11, 2014|      100|      7.5|            1,869|           274|              354|
|          Casablanca|[Drama, Romance, ...| January 23, 1943|      100|      8.9|              360|            11|               10|
|         Rear Window| [Mystery, Thriller]|September 1, 1954|      100|      8.8|              365|            15|               10|
|       The Godfather|[Drama, Thriller,...|   March 11, 1972|      10

In [13]:
from pyspark.sql.functions import col
df = (sparkdf
       .withColumn("Metascore", col("Metascore").cast("float"))
       .withColumn("Userscore", col("Userscore").cast("float"))
       .withColumn("UserScorePositive", col("UserScorePositive").cast("float"))
       .withColumn("UserScoreMixed", col("UserScoreMixed").cast("float"))
       .withColumn("UserScoreNegative", col("UserScoreNegative").cast("float"))
       .dropna()
       )

In [14]:
df

DataFrame[title: string, genres: array<string>, date: string, Metascore: float, Userscore: float, UserScorePositive: float, UserScoreMixed: float, UserScoreNegative: float]

In [15]:
from pyspark.sql.functions import col


In [16]:
df1= df.withColumn("PerPostive", (col("UserScorePositive") / col("UserScorePositive")+col("UserScoreMixed")+col('UserScoreNegative')))

In [17]:
df1.show(3)

+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+----------+
|            title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|PerPostive|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+----------+
|Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|            152.0|           4.0|              9.0|      14.0|
|       Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|            360.0|          11.0|             10.0|      22.0|
|      Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8|            365.0|          15.0|             10.0|      26.0|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+----------+
only showing 

In [24]:
df1.printSchema()

root
 |-- title: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date: string (nullable = true)
 |-- Metascore: float (nullable = true)
 |-- Userscore: float (nullable = true)
 |-- UserScorePositive: float (nullable = true)
 |-- UserScoreMixed: float (nullable = true)
 |-- UserScoreNegative: float (nullable = true)
 |-- PerPostive: double (nullable = true)



In [31]:
df11=df1.drop(df1.UserScorePositive)

df11.show()

+--------------------+--------------------+-----------------+---------+---------+--------------+-----------------+----------+
|               title|              genres|             date|Metascore|Userscore|UserScoreMixed|UserScoreNegative|PerPostive|
+--------------------+--------------------+-----------------+---------+---------+--------------+-----------------+----------+
|   Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|           4.0|              9.0|      14.0|
|          Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|          11.0|             10.0|      22.0|
|         Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8|          15.0|             10.0|      26.0|
|         Intolerance|    [Drama, History]|September 5, 1916|     99.0|      7.4|           1.0|              4.0|       6.0|
|         City Lights|[Drama, Comedy, R...|    March 7, 1931|     99.0|      8.9|           1.0|              3.0|    

In [18]:
df2 = df.withColumn("Sum", col("UserScorePositive")+ col("UserScoreMixed")+col('UserScoreNegative'))

In [19]:
df2.show(3)

+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+
|            title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|  Sum|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+
|Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|            152.0|           4.0|              9.0|165.0|
|       Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|            360.0|          11.0|             10.0|381.0|
|      Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8|            365.0|          15.0|             10.0|390.0|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+
only showing top 3 rows



In [20]:
Nega = df2.withColumn("Negative", col("UserScoreNegative") / col("Sum"))
Nega.show(2)

+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+--------------------+
|            title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|  Sum|            Negative|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+--------------------+
|Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|            152.0|           4.0|              9.0|165.0| 0.05454545454545454|
|       Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|            360.0|          11.0|             10.0|381.0|0.026246719160104987|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+--------------------+
only showing top 2 rows



In [33]:
dfNega=Nega.drop('UserScoreNegative','UserScoreMixed','UserScorePositive','Sum')
dfNega.show()

+--------------------+--------------------+-----------------+---------+---------+--------------------+
|               title|              genres|             date|Metascore|Userscore|            Negative|
+--------------------+--------------------+-----------------+---------+---------+--------------------+
|   Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6| 0.05454545454545454|
|          Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|0.026246719160104987|
|         Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8| 0.02564102564102564|
|         Intolerance|    [Drama, History]|September 5, 1916|     99.0|      7.4|  0.2222222222222222|
|         City Lights|[Drama, Comedy, R...|    March 7, 1931|     99.0|      8.9| 0.02830188679245283|
| Singin' in the Rain|[Comedy, Romance,...|   April 11, 1952|     99.0|      8.7|0.026737967914438502|
|           Notorious|[Drama, Thriller,...|September 6, 1946|    100.0|  

In [21]:
Posi = df2.withColumn(
    "Positive", col("UserScorePositive") / col("Sum")
    
    )
Posi.show(3)

+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+------------------+
|            title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|  Sum|          Positive|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+------------------+
|Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|            152.0|           4.0|              9.0|165.0|0.9212121212121213|
|       Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|            360.0|          11.0|             10.0|381.0|0.9448818897637795|
|      Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8|            365.0|          15.0|             10.0|390.0|0.9358974358974359|
+-----------------+--------------------+-----------------+---------+--

In [60]:
dfPosi=Posi.drop('UserScorePositive','UserScoreMixed','UserScoreNegative','Sum','date')
dfPosi.show()

+--------------------+--------------------+---------+---------+------------------+
|               title|              genres|Metascore|Userscore|          Positive|
+--------------------+--------------------+---------+---------+------------------+
|   Three Colors: Red|[Drama, Mystery, ...|    100.0|      8.6|0.9212121212121213|
|          Casablanca|[Drama, Romance, ...|    100.0|      8.9|0.9448818897637795|
|         Rear Window| [Mystery, Thriller]|    100.0|      8.8|0.9358974358974359|
|         Intolerance|    [Drama, History]|     99.0|      7.4|0.7222222222222222|
|         City Lights|[Drama, Comedy, R...|     99.0|      8.9|0.9622641509433962|
| Singin' in the Rain|[Comedy, Romance,...|     99.0|      8.7|0.9037433155080213|
|           Notorious|[Drama, Thriller,...|    100.0|      8.0|0.8648648648648649|
|    Some Like It Hot|   [Comedy, Romance]|     98.0|      8.4|0.9032258064516129|
|The Treasure of t...|[Adventure, Drama...|     98.0|      8.5|0.9207920792079208|
|   

In [22]:
Mixed = df2.withColumn(
   
    "Mixed", col("UserScoreMixed") / col("Sum")
    )
Mixed .show(3)

+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+--------------------+
|            title|              genres|             date|Metascore|Userscore|UserScorePositive|UserScoreMixed|UserScoreNegative|  Sum|               Mixed|
+-----------------+--------------------+-----------------+---------+---------+-----------------+--------------+-----------------+-----+--------------------+
|Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|            152.0|           4.0|              9.0|165.0|0.024242424242424242|
|       Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|            360.0|          11.0|             10.0|381.0|0.028871391076115485|
|      Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8|            365.0|          15.0|             10.0|390.0|0.038461538461538464|
+-----------------+--------------------+-----------------+

In [36]:
dfMixed=Mixed.drop('UserScorePositive','UserScoreMixed','UserScoreNegative')
dfMixed.show()

+--------------------+--------------------+-----------------+---------+---------+-----+--------------------+
|               title|              genres|             date|Metascore|Userscore|  Sum|               Mixed|
+--------------------+--------------------+-----------------+---------+---------+-----+--------------------+
|   Three Colors: Red|[Drama, Mystery, ...|November 23, 1994|    100.0|      8.6|165.0|0.024242424242424242|
|          Casablanca|[Drama, Romance, ...| January 23, 1943|    100.0|      8.9|381.0|0.028871391076115485|
|         Rear Window| [Mystery, Thriller]|September 1, 1954|    100.0|      8.8|390.0|0.038461538461538464|
|         Intolerance|    [Drama, History]|September 5, 1916|     99.0|      7.4| 18.0| 0.05555555555555555|
|         City Lights|[Drama, Comedy, R...|    March 7, 1931|     99.0|      8.9|106.0|0.009433962264150943|
| Singin' in the Rain|[Comedy, Romance,...|   April 11, 1952|     99.0|      8.7|187.0| 0.06951871657754011|
|           Notorio

In [70]:
from pyspark.sql.functions import *

In [83]:
dfMixed = dfMixed.alias('dfMixed')
dfPosi = dfPosi.alias('dfPosi')

newdf =dfMixed.join(dfPosi, dfMixed.title == dfPosi.title).select(dfMixed['*'],dfPosi['Positive'])
newdf.show(2)

+------------------+--------------------+-------------+---------+---------+----+-------------------+------------------+
|             title|              genres|         date|Metascore|Userscore| Sum|              Mixed|          Positive|
+------------------+--------------------+-------------+---------+---------+----+-------------------+------------------+
|          Deep Red|[Mystery, Thrille...|June 11, 1976|     89.0|      8.2|23.0|0.17391304347826086|0.8260869565217391|
|Living in Oblivion|     [Drama, Comedy]|July 21, 1995|     81.0|      7.0| 5.0|                0.0|               0.8|
+------------------+--------------------+-------------+---------+---------+----+-------------------+------------------+
only showing top 2 rows



In [84]:
newdf = newdf.alias('newdf')
dfNega = dfNega.alias('dfNega')

joinnew = newdf.join(dfNega, dfNega.date == newdf.date).select(newdf['*'],dfNega['Negative'])
joinnew.show()

+--------------------+--------------------+-----------------+---------+---------+----+--------------------+------------------+--------------------+
|               title|              genres|             date|Metascore|Userscore| Sum|               Mixed|          Positive|            Negative|
+--------------------+--------------------+-----------------+---------+---------+----+--------------------+------------------+--------------------+
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1|35.0| 0.45714285714285713|0.5428571428571428| 0.18823529411764706|
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1|35.0| 0.45714285714285713|0.5428571428571428|  0.1320754716981132|
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1|35.0| 0.45714285714285713|0.5428571428571428|  0.6745283018867925|
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1|35.0| 0.45714285714285713|0.542

In [87]:
totdf= joinnew.drop('Sum') 
totdf.show()

+--------------------+--------------------+-----------------+---------+---------+--------------------+------------------+--------------------+
|               title|              genres|             date|Metascore|Userscore|               Mixed|          Positive|            Negative|
+--------------------+--------------------+-----------------+---------+---------+--------------------+------------------+--------------------+
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1| 0.45714285714285713|0.5428571428571428| 0.18823529411764706|
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1| 0.45714285714285713|0.5428571428571428|  0.1320754716981132|
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1| 0.45714285714285713|0.5428571428571428|  0.6745283018867925|
| The Invisible Woman|[Biography, Drama...|December 25, 2013|     75.0|      7.1| 0.45714285714285713|0.5428571428571428| 0.06890756302521009|

In [103]:
#Normalized MetaScore
normaltot=totdf.withColumn('NormalizedMetaScore', (col('Metascore')/10))
normaldf = normaltot.drop('Metascore')
normaldf.show()
type(normaldf)

+--------------------+--------------------+-----------------+---------+-------------------+------------------+--------------------+-------------------+
|               title|              genres|             date|Userscore|              Mixed|          Positive|            Negative|NormalizedMetaScore|
+--------------------+--------------------+-----------------+---------+-------------------+------------------+--------------------+-------------------+
|            Deep Red|[Mystery, Thrille...|    June 11, 1976|      8.2|0.17391304347826086|0.8260869565217391|                 0.0|                8.9|
|  Living in Oblivion|     [Drama, Comedy]|    July 21, 1995|      7.0|                0.0|               0.8|                 0.2|                8.1|
|          Starred Up|             [Drama]|  August 27, 2014|      8.1|0.07368421052631578|0.9052631578947369| 0.17525773195876287|                8.1|
|          Starred Up|             [Drama]|  August 27, 2014|      8.1|0.073684210526315

pyspark.sql.dataframe.DataFrame

In [150]:
normaldf.show(2)
type(normaldf)


+------------------+--------------------+-------------+---------+-------------------+------------------+--------+-------------------+
|             title|              genres|         date|Userscore|              Mixed|          Positive|Negative|NormalizedMetaScore|
+------------------+--------------------+-------------+---------+-------------------+------------------+--------+-------------------+
|          Deep Red|[Mystery, Thrille...|June 11, 1976|      8.2|0.17391304347826086|0.8260869565217391|     0.0|                8.9|
|Living in Oblivion|     [Drama, Comedy]|July 21, 1995|      7.0|                0.0|               0.8|     0.2|                8.1|
+------------------+--------------------+-------------+---------+-------------------+------------------+--------+-------------------+
only showing top 2 rows



pyspark.sql.dataframe.DataFrame

In [153]:
normaldf.write.json('normalized_data')


AnalysisException: ignored

In [145]:

#dfnor1 = normaldf.select('*',round('Positive',2)).show()
#normaldf.select('*',round('Mixed',2))
#normaldf.select('*',round('Negative',2))



+--------------------+--------------------+-----------------+---------+-------------------+------------------+--------------------+-------------------+------------------+
|               title|              genres|             date|Userscore|              Mixed|          Positive|            Negative|NormalizedMetaScore|round(Positive, 2)|
+--------------------+--------------------+-----------------+---------+-------------------+------------------+--------------------+-------------------+------------------+
|            Deep Red|[Mystery, Thrille...|    June 11, 1976|      8.2|0.17391304347826086|0.8260869565217391|                 0.0|                8.9|              0.83|
|  Living in Oblivion|     [Drama, Comedy]|    July 21, 1995|      7.0|                0.0|               0.8|                 0.2|                8.1|               0.8|
|          Starred Up|             [Drama]|  August 27, 2014|      8.1|0.07368421052631578|0.9052631578947369| 0.17525773195876287|              

AttributeError: ignored